In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns

from GoogleNews import GoogleNews
import newspaper
from newspaper import Article
from newspaper import Config
import nltk
import openpyxl
from gensim import corpora, models, matutils
from collections import defaultdict
from time import sleep
import datetime


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from textblob import TextBlob, Word
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer

nltk_stops = stopwords.words()
nltk.download('punkt')

%matplotlib inline

# config will allow us to access the specified url 
# for which we are #not authorized. Sometimes we may 
# get 403 client error while parsing #the link to 
# download the article.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
search_term = str(input())

def web_search(self):
    # Set up a list of user agents to iterate through
    user_agents = [
        ('Mozilla/5.0'),  # firefox
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15'),  # safari
        ('Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0'),  # firefox
        ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.110 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) Gecko/20100101 Firefox/55.0'),  # firefox
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.91 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.108 Safari/537.36'),  # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'), # chrome
        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'), # firefox
        ]
    # user_agent = 'Mozilla/5.0'
    
    # # # While loop
    # # Finding the length of the user agent string list  
    # length = len(user_agents)
    # i = 0
    # # While loop to iterate through list
    # while i < length:
    #     # action to complete and iterate with different user agent strings

    #     i += 1

    list = []

    try:
        for user_agent in user_agents:
            # Configure the browser agent
            config = Config()
            config.browser_user_agent = user_agent[0:]

            # Set up search term(s) and date range
            # Create variables for data storage results
            googlenews = GoogleNews(start='05/01/2021', end='09/11/2021')
            googlenews.search(search_term)
            result = googlenews.result()
            df = pd.DataFrame(result)
            print(df.head())

            for i in range(1,51):
                googlenews.getpage(i)
                result = googlenews.result()
                df = pd.DataFrame(result)
                sleep(5)
        
            for ind in df.index:
                dict = {}

                # Use the defined User Agent string to bypass Error: 403 codes
                article = Article(df['link'][ind], config=config)

                # Download the article return
                article.download()

                # Parse the article return
                article.parse()
                article.nlp()

                # Create new columns in the DataFrame to store data
                dict['Date']=df['date'][ind]
                dict['Media']=df['media'][ind]
                dict['Title']=article.title
                dict['Article']=article.text
                dict['Summary']=article.summary
                # Append to list
                list.append(dict)
                sleep(5)
            # Create a new DataFrame for the appended list
            news_df=pd.DataFrame(list)
            # Output to excel
            news_df.to_excel("articles.xlsx")
    except:
        for user_agent in user_agents:
            # Configure the browser agent
            config = Config()
            config.browser_user_agent = user_agent[1:]

            # Set up search term(s) and date range
            # Create variables for data storage results
            googlenews = GoogleNews(start='05/01/2021', end='09/11/2021')
            googlenews.search(search_term)
            result = googlenews.result()
            print(df.head())

            for i in range(1,51):
                googlenews.getpage(i)
                result = googlenews.result()
                df = pd.DataFrame(result)
                sleep(5)
        
            for ind in df.index:
                dict = {}

                # Use the defined User Agent string to bypass Error: 403 codes
                article = Article(df['link'][ind], config=config)

                # Download the article return
                article.download()

                # Parse the article return
                article.parse()
                article.nlp()

                # Create new columns in the DataFrame to store data
                dict['Date']=df['date'][ind]
                dict['Media']=df['media'][ind]
                dict['Title']=article.title
                dict['Article']=article.text
                dict['Summary']=article.summary
                # Append to list
                list.append(dict)
                sleep(5)
            # Create a new DataFrame for the appended list
            news_df=pd.DataFrame(list)
            # Output to excel
            news_df.to_excel("articles.xlsx")




    # for user_agent in user_agents:
    #     # Configure the browser user agent
    #     config = Config()
    #     config.browser_user_agent = user_agent

    #     # Set up search term(s) and date range
    #     # # Create variables for data storage results
    #     googlenews = GoogleNews(start='05/01/2021', end='09/11/2021')
    #     googlenews.search(search_term)
    #     result = googlenews.result()
    #     df = pd.DataFrame(result)
    #     print(df.head())

    #     for i in range(1,51):
    #         googlenews.getpage(i)
    #         result = googlenews.result()
    #         df = pd.DataFrame(result)
    #         sleep(5)
        
    #     for ind in df.index:
    #         dict = {}

    #         # Use the defined User Agent string to bypass Error: 403 codes
    #         article = Article(df['link'][ind], config=config)

    #         # Download the article return
    #         article.download()

    #         # Parse the article return
    #         article.parse()
    #         article.nlp()

    #         # Create new columns in the DataFrame to store data
    #         dict['Date']=df['date'][ind]
    #         dict['Media']=df['media'][ind]
    #         dict['Title']=article.title
    #         dict['Article']=article.text
    #         dict['Summary']=article.summary
    #         # Append to list
    #         list.append(dict)
    #         sleep(5)
    # # Create a new DataFrame for the appended list
    # news_df=pd.DataFrame(list)
    # # Output to excel
    # news_df.to_excel("articles.xlsx")
web_search(search_term)

HTTP Error 429: Too Many Requests
Empty DataFrame
Columns: []
Index: []
'NoneType' object is not iterable
'NoneType' object is not iterable
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
Empty DataFrame
Columns: []
Index: []
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


## Query the data and save to a DataFrame

In [2]:
# user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
# user_agent = {'User-agent': 'Mozilla/5.0'}
user_agent = 'Mozilla/5.0'
# user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent

In [3]:
# Set up search terms and date range
# Create variables for data storage results
googlenews=GoogleNews(start='05/01/2021',end='09/11/2021')
googlenews.search('Afghanistan'); sleep(1)
result=googlenews.result(); sleep(1)
df=pd.DataFrame(result)
print(df.head())

HTTP Error 429: Too Many Requests
Empty DataFrame
Columns: []
Index: []


In [4]:
# Iterate through Google News, get the page
# results, and store them in a DataFrame
for i in range(2,20):
    googlenews.getpage(i) #; sleep(2)
    result = googlenews.result()
    df=pd.DataFrame(result)
    sleep(1)

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


C:\Users\josep\AppData\Local\Programs\Python\Python310\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


In [5]:
list=[]

for ind in df.index:
    dict={}
    # Use the defined User Agent string to bypass Error: 403 codes
    article = Article(df['link'][ind],config=config)
    # Download the article return
    article.download()
    # Parse the article return
    article.parse()
    article.nlp()
    # Create new columns in the DataFrame to store data
    dict['Date']=df['date'][ind]
    dict['Media']=df['media'][ind]
    dict['Title']=article.title
    dict['Article']=article.text
    dict['Summary']=article.summary
    # Append to list
    list.append(dict)
    sleep(1)
# Create a new DataFrame for the appended list
news_df=pd.DataFrame(list)
# Output to excel
news_df.to_excel("articles.xlsx")

## Store the data and inspect the output for errors

In [6]:
# Read the excel file and store into a variable
data = pd.read_excel('./articles.xlsx')

# Check the file and data
data.head()

,Unnamed: 0,Date,Media,Title,Article,Summary
0,0,"Aug 31, 2021",Wall Street Journal,A Dishonest Afghanistan Accounting,"American Presidents must make hard decisions, ...","American Presidents must make hard decisions, ..."
1,1,"Aug 31, 2021",NPR,"With The Americans Gone, Afghanistan Enters It...","With The Americans Gone, Afghanistan Enters It...","""Congratulations to Afghanistan,"" the group's ..."
2,2,"Aug 31, 2021",The New York Times,Opinion | Joe Biden’s Critics Lost Afghanistan,A month ago I thought I was a cynic about our ...,A month ago I thought I was a cynic about our ...
3,3,"Aug 31, 2021",CNN,The last US military planes have left Afghanis...,(CNN) The last US military planes have left Af...,(CNN) The last US military planes have left Af...
4,4,"Aug 31, 2021",BBC,Afghanistan: Last US military flight departs e...,If you are reading this page and can't see the...,If you are reading this page and can't see the...


In [7]:
data.shape

(20, 6)

In [8]:
# Rename columns names appropriately
data.rename(columns={'Unnamed: 0' : 'Index'}, inplace=True)
# Set the Index column as the data index
data.set_index('Index', inplace=True)
data.head()

,Date,Media,Title,Article,Summary
Index,,,,,
0,"Aug 31, 2021",Wall Street Journal,A Dishonest Afghanistan Accounting,"American Presidents must make hard decisions, ...","American Presidents must make hard decisions, ..."
1,"Aug 31, 2021",NPR,"With The Americans Gone, Afghanistan Enters It...","With The Americans Gone, Afghanistan Enters It...","""Congratulations to Afghanistan,"" the group's ..."
2,"Aug 31, 2021",The New York Times,Opinion | Joe Biden’s Critics Lost Afghanistan,A month ago I thought I was a cynic about our ...,A month ago I thought I was a cynic about our ...
3,"Aug 31, 2021",CNN,The last US military planes have left Afghanis...,(CNN) The last US military planes have left Af...,(CNN) The last US military planes have left Af...
4,"Aug 31, 2021",BBC,Afghanistan: Last US military flight departs e...,If you are reading this page and can't see the...,If you are reading this page and can't see the...


## Begin using NLP to process article returns

In [9]:
# custom_stop_words = list(ENGLISH_STOP_WORDS)

In [10]:
# article = data.Article
# article

# article = pd.DataFrame(data, columns='Article')
# article

article = pd.DataFrame(data.Article)
article['Article']

Index
0     American Presidents must make hard decisions, ...
1     With The Americans Gone, Afghanistan Enters It...
2     A month ago I thought I was a cynic about our ...
3     (CNN) The last US military planes have left Af...
4     If you are reading this page and can't see the...
5     U.S. Army soldiers assigned to the 82nd Airbor...
6     That the Taliban is back is power in Afghanist...
7     In this image made through a night vision scop...
8     It was always a fool’s errand.\n\nThe botched ...
9     France, which evacuated more than 2,800 people...
10    For the last few weeks, Americans have been st...
11    In this image made through a night vision scop...
12    In a statement released on Tuesday, Mr. Guterr...
13    It was always a fool’s errand.\n\nThe botched ...
14    France, which evacuated more than 2,800 people...
15    In the final days of the mission, a suicide bo...
16    The Pentagon announced Monday that the last U....
17    When comparing the Taliban with the 

In [11]:
# Replace NaN with ''
article.fillna('', inplace=True)

In [12]:
article.head()

,Article
Index,
0,"American Presidents must make hard decisions, ..."
1,"With The Americans Gone, Afghanistan Enters It..."
2,A month ago I thought I was a cynic about our ...
3,(CNN) The last US military planes have left Af...
4,If you are reading this page and can't see the...


### 3. Use CountVectorizer to transform text, taking out the stopwords.

In [13]:
stop_words = set(stopwords.words('english'))

vectorizer = CountVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(article['Article'])

### 4. Extract the tokens that remain after stopword removal.

The `.vocabulary_` attribute of the vectorizer contains a dictionary of terms. There is also the built-in function `.get_feature_names()` which will extract the column names.

In [14]:
# sorted(vectorizer.vocabulary_, reverse=False)[:]
vectorizer.vocabulary_

{'american': 189,
 'presidents': 1796,
 'must': 1557,
 'make': 1435,
 'hard': 1121,
 'decisions': 652,
 'inclined': 1212,
 'support': 2274,
 'overseas': 1660,
 'national': 1567,
 'interest': 1254,
 'president': 1795,
 'biden': 321,
 'defiant': 667,
 'accusatory': 95,
 'defense': 666,
 'tuesday': 2434,
 'afghanistan': 138,
 'withdrawal': 2584,
 'execution': 903,
 'dishonest': 731,
 'lacking': 1329,
 'self': 2095,
 'reflection': 1912,
 'accountability': 92,
 'unworthy': 2481,
 'sacrifices': 2042,
 'americans': 190,
 'made': 1423,
 'conflict': 537,
 'charitable': 451,
 'interpretation': 1258,
 'mr': 1547,
 'really': 1893,
 'believes': 314,
 'particular': 1683,
 'war': 2536,
 'general': 1049,
 'defend': 665,
 'uncharitable': 2453,
 'view': 2512,
 'advisers': 130,
 'decided': 648,
 'way': 2548,
 'debacle': 640,
 'lie': 1380,
 'blame': 330,
 'everyone': 886,
 'else': 809,
 'claim': 474,
 'defeat': 662,
 'victory': 2509,
 'neither': 1581,
 'one': 1628,
 'reassuring': 1895,
 'character': 448,


In [15]:
dict(sorted(vectorizer.vocabulary_))

TypeError: 'dict' object is not callable

In [ ]:
# word_tokens = word_tokenize(

In [ ]:
vectorizer.get_feature_names_out()

array(['000', '10', '100', ..., 'zone', 'zones', 'zulus'], dtype=object)

In [ ]:
docs = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
docs.sum()

C:\Users\josep\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


000       15
10         1
100        6
105        1
11         4
          ..
zenith     1
zero       1
zone       2
zones      1
zulus      1
Length: 2652, dtype: int64

### 6. Setup the vocabulary dictionary

First we need to setup the vocabulary.  Gensim's LDA expects our vocabulary to be in a format where the dictionary keys are the column indices and the values are the words themselves.

Create this dictionary below.

In [ ]:
# This is the fastest way to swap a dictionary key / value order.  
# This is the format gensim LDA expects it's vocabulary.
vocab = {v: k for k, v in vectorizer.vocabulary_.items()}
vocab

{372: 'capabilities',
 2466: 'united',
 2248: 'states',
 835: 'envisioned',
 2612: 'within',
 117: 'afghanistan',
 1439: 'longer',
 1781: 'possible',
 1644: 'officials',
 2642: 'yet',
 2051: 'said',
 1353: 'kind',
 825: 'engagement',
 1524: 'might',
 2092: 'seek',
 1609: 'new',
 2321: 'taliban',
 1072: 'government',
 2227: 'sprawling',
 798: 'embassy',
 1338: 'kabul',
 1416: 'likely',
 1942: 'remain',
 2155: 'shuttered',
 1595: 'near',
 2347: 'term',
 1651: 'one',
 1643: 'official',
 1548: 'monday',
 1529: 'military',
 1946: 'remains',
 985: 'focused',
 856: 'evacuating',
 157: 'americans',
 141: 'allies',
 2271: 'stranded',
 116: 'afghan',
 376: 'capital',
 2297: 'sunday',
 617: 'day',
 379: 'captured',
 2529: 'volant',
 1506: 'media',
 2442: 'uk',
 1457: 'ltd',
 1700: 'parent',
 494: 'company',
 1304: 'iran',
 1285: 'international',
 2434: 'tv',
 1380: 'launched',
 135: 'aitv',
 214: 'aspires',
 2114: 'serve',
 33: '24',
 1610: 'news',
 1606: 'network',
 1845: 'provide',
 1937: 'reli

### 7. Create a token to id mapping with gensim's `corpora.Dictionary`

This dictionary class is a more standard way to work with with gensim models. There are a few standard steps we should go through:

**7.1. Count the frequency of words.**

We can do this easily with the python `defaultdict(int)`, which doesn't require us to already have the key in the dictionary to be able to add to it:

```python
frequency = defaultdict(int)

for text in documents:
    for token in text.split():
        frequency[token] += 1
```

In [ ]:
articles = article['Article']
articles

Index
0                                                      
1                                                      
2                                                      
3     Now, most if not all of the capabilities the U...
4     On Sunday, the day the Taliban captured Afghan...
5                                                      
6                                                      
7                                                      
8     Afghan policemen stand guard next to Indian an...
9     Choose which Defense.gov products you want del...
10    In recent weeks, Russia has advanced more than...
11                                                     
12    Afghan journalists and rights activists have c...
13    Listen to this episode of The Times: Apple | S...
14    The United States is preparing to withdraw fro...
15    As U.S. President Joe Biden continues his Euro...
16    Finland has had a military presence in Afghani...
17    Gen. Mark A. Milley speaks at a Chan

In [ ]:
articles.shape

(29,)

In [ ]:
frequency = defaultdict(int)

for text in articles:
    for token in text.split():
        frequency[token] += 1
        
frequency

defaultdict(int,
            {'Now,': 3,
             'most': 13,
             'if': 16,
             'not': 38,
             'all': 26,
             'of': 288,
             'the': 647,
             'capabilities': 2,
             'United': 18,
             'States': 12,
             'had': 51,
             'envisioned': 1,
             'within': 2,
             'Afghanistan': 61,
             'are': 47,
             'no': 14,
             'longer': 1,
             'possible.': 4,
             'While': 2,
             'officials': 10,
             'have': 72,
             'yet': 2,
             'said': 43,
             'what': 12,
             'kind': 3,
             'engagement': 2,
             'they': 40,
             'might': 9,
             'seek,': 1,
             'any,': 1,
             'with': 78,
             'new': 5,
             'Taliban': 43,
             'government,': 3,
             'sprawling': 1,
             'U.S.': 75,
             'Embassy': 4,
             'in': 2

**7.2 Remove any words that only appear once, or appear in the stopwords.**

Iterate through the documents and only keep useful words/tokens.

In [ ]:
texts = [[token for token in text.split() if frequency[token] > 1 and token not in nltk_stops]
          for text in articles]

texts

[[],
 [],
 [],
 ['Now,',
  'capabilities',
  'United',
  'States',
  'within',
  'Afghanistan',
  'possible.',
  'While',
  'officials',
  'yet',
  'said',
  'kind',
  'engagement',
  'might',
  'new',
  'Taliban',
  'government,',
  'U.S.',
  'Embassy',
  'Kabul',
  'likely',
  'remain',
  'near',
  'For',
  'now,',
  'official',
  'said',
  'U.S.',
  'military',
  'remains',
  'evacuating',
  'Americans',
  'allies',
  'Afghan'],
 ['On',
  'day',
  'Taliban',
  'captured',
  'Afghanistan’s',
  'capital',
  'Kabul,',
  'Iran',
  'International',
  'launched',
  'Afghanistan',
  'International',
  'TV',
  'AITV',
  'aspires',
  '24/7',
  'international',
  'news',
  'network',
  'provide',
  'information',
  'audiences',
  'protect',
  'media',
  'country.',
  'With',
  'Afghanistan',
  'chaotic',
  'US',
  'military',
  'withdrawal,',
  'AITV',
  'aspires',
  'voice',
  'tell',
  'story',
  'people',
  'Afghanistan,',
  'director',
  'tells',
  'Iran',
  'International',
  'He',
  're

**7.3 Create the `corpora.Dictionary` object with the retained tokens.**

In [ ]:
# Create gensim dictionary object
dictionary = corpora.Dictionary(texts)
dictionary

**7.4 Use the `dictionary.doc2bow()` function to convert the texts to bag-of-word representations.**

In [ ]:
# Create corpus matrix
corpus = [dictionary.doc2bow(text) for text in texts]
corpus

[[],
 [],
 [],
 [(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 2),
  (30, 1),
  (31, 1)],
 [(1, 4),
  (8, 3),
  (10, 1),
  (20, 1),
  (26, 1),
  (29, 1),
  (32, 2),
  (33, 8),
  (34, 2),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 4),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 2),
  (75, 2),
  (76, 1),
  (77, 2),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 3),
  (84,

**Why should we use this process?**

The main advantage is that this dictionary object has quick helper functions.

However, there are also some major performance advantages if you ever want to save your model to a file, then load it at a later time.  Tokenizations can take a while to be computed, especially when your text files are quite large. You can save these post-computed dictionary items to file, then load them from disk later which is quite a bit faster.  Also, it's possible to add new documents to your corpus without having to re-tokenize your entire set.  This is great for online systems that can take new documents on demmand.  

As you work with larger datasets with text, this is a much better way to handle LDA and other Gensim models from a performance point of view.

### 8. Set up the LDA model

We can create the gensim LDA model object like so:

```python
lda = models.LdaModel(
    # supply our sparse predictor matrix wrapped in a matutils.Sparse2Corpus object
    matutils.Sparse2Corpus(X, documents_columns=False),
    # or alternatively use the corpus object created with the dictionary in the previous frame!
    # corpus,
    # The number of topics we want:
    num_topics  =  3,
    # how many passes over the vocabulary:
    passes      =  20,
    # The id2word vocabulary we made ourselves
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)
```

In [ ]:
lda = models.LdaModel(
    matutils.Sparse2Corpus(X, documents_columns=False),
    num_topics  =  3,
    passes      =  20,
    id2word     =  vocab
)

### 9. Look at the topics

The model has a `.print_topics` function that accepts the number of topics to print and number of words per topic. The number before the word is the probability of occurance for that word in the topic.

In [ ]:
lda.print_topics(num_topics=3, num_words=5)

[(0,
  '0.015*"afghanistan" + 0.011*"said" + 0.007*"taliban" + 0.006*"people" + 0.006*"airport"'),
 (1,
  '0.013*"said" + 0.012*"afghan" + 0.011*"afghanistan" + 0.010*"taliban" + 0.008*"forces"'),
 (2,
  '0.019*"nato" + 0.006*"afghanistan" + 0.005*"said" + 0.005*"question" + 0.005*"europe"')]

### 10. Get the topic scores for a document

The `.get_document_topics` function accepts a bag-of-words representation for a document and returns the scores for each topic.

In [ ]:
lda.get_document_topics(dictionary.doc2bow(texts[2]))

[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]

### 11. Label and visualize the topics

Lets come up with some high level labels. This is the subjective part of LDA. What do the word probabilties that represent topics mean?  Let's make some up.

Plot a heatmap of the topic probabilities for each of the documents.

In [ ]:
topics_labels = {
   0: "biden",
   1: "taliban",
   2: "afghanistan"
}

In [ ]:
doc_topics = [lda.get_document_topics(doc) for doc in corpus]

topic_data = []

for document_id, topics in enumerate(doc_topics):
    
    document_topics = []
    
    for topic, probability in topics:
       
        topic_data.append({
            'document_id':  document_id,
            'topic_id':     topic,
            'topic':        topics_labels[topic],
            'probability':  probability
        })

topics_df = pd.DataFrame(topic_data)
topics_df.pivot_table(values="probability", index=["document_id", "topic"]).T

document_id          0                               1                       \
topic       afghanistan     biden   taliban afghanistan     biden   taliban   
probability    0.333333  0.333333  0.333333    0.333333  0.333333  0.333333   

document_id          2                               3   ...        25  \
topic       afghanistan     biden   taliban afghanistan  ...   taliban   
probability    0.333333  0.333333  0.333333    0.086788  ...  0.414109   

document_id          26                              27                      \
topic       afghanistan     biden   taliban afghanistan     biden   taliban   
probability    0.185034  0.502535  0.312431    0.226288  0.502153  0.271559   

document_id          28                      
topic       afghanistan     biden   taliban  
probability    0.032787  0.729138  0.238075  

[1 rows x 87 columns]

In [ ]:
doc_topics = [lda.get_document_topics(doc) for doc in corpus]

doc_topic_probabilities = []

for document in doc_topics:
    
    single_document = []
    
    for topic, probablity in document:
        
        single_document.append(probablity)
        
    doc_topic_probabilities.append(single_document)
    
# docs_topics = pd.DataFrame(doc_topic_probabilities.index(1))
docs_topics = pd.DataFrame(doc_topic_probabilities)
sns.heatmap(docs_topics)

NameError: name 'corpus' is not defined

### 12. Fit an LDA model with sklearn

Sklearn's LDA model is in the decomposition submodule:

```python
from sklearn.decomposition import LatentDirichletAllocation
```

One of the greatest benefits of the sklearn implementation is that it comes with the familiar `.fit()`, `.transform()` and `.fit_transform()` methods.

**12.1 Initialize and fit an sklearn LDA with `n_topics=3` on our output from the CountVectorizer.**

In [ ]:
lda = LatentDirichletAllocation(n_components=3)
lda.fit(X)

LatentDirichletAllocation(n_components=3)

**12.2 Print out the topic-word distributions using the `.components_` attribute.**

Each row of this matrix represents a topic, and the columns are the words. (These are not probabilities).

In [ ]:
lda.components_

array([[4.34376888, 1.33332381, 0.33342501, ..., 0.3333416 , 0.33334244,
        1.33332709],
       [9.35074126, 0.33333795, 3.33661845, ..., 1.33334988, 0.33333856,
        0.33333626],
       [2.30548986, 0.33333824, 3.32995654, ..., 1.33330852, 1.333319  ,
        0.33333664]])

**12.3 Use the `.transform()` method to convert the matrix into the topic scores.**

These are the document-topic distributions.

In [ ]:
lda.transform(X)

array([[3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [9.84387071e-03, 9.81854827e-03, 9.80337581e-01],
       [1.91494185e-03, 2.02941513e-03, 9.96055643e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [9.93590756e-01, 3.16157590e-03, 3.24766840e-03],
       [4.67815743e-02, 9.10227690e-01, 4.29907361e-02],
       [1.95723263e-02, 2.04624885e-02, 9.59965185e-01],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [9.61972202e-01, 1.84318490e-02, 1.95959488e-02],
       [3.86170629e-03, 3.92152223e-03, 9.92216771e-01],
       [9.83996052e-04, 1.00977212e-03, 9.98006232e-01],
       [4.79830580e-04, 4.86687732e-04, 9.99033482e-01],
       [1.76437239e-03, 9.96224064e-01, 2.01156358e-03],
       [6.66611983e-04, 6.89574